# System

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Append inline properties to system configuration

In [ ]:
%%luminesce

-- =====================================================================================
-- Description:
-- In this query, make properties available to Luminesce.
-- This approach appends rather than overwrites the current inline property configuration
-- See the following page for further details:
-- https://support.lusid.com/knowledgebase/article/KA-01702/en-us
-- ======================================================================================

-- 1. Get the contents of the current instrument provider factory

@@currentFileContent = 
select Content from Sys.File where Name = 'instrumentequityproviderfactory';

@currentFileLines = 
select Value from Tools.Split
where Original = @@currentFileContent
    and Delimiters = '
' -- newline
    and [Index] <> 1 -- skip header
    and Value is not null
    and Trim(Value, ' ') <> '';

@splitFileLines =
select OriginalIndex, 
    case when [Index] = 1 then Value else null end as PropertyKey, 
    case when [Index] = 2 then Value else null end as DataType, 
    case when [Index] = 3 then Value else null end as Alias,
    case when [Index] = 4 then Value else null end as [Description]
from Tools.Split
where Original in @currentFileLines and Delimiters = ',';

@currentProperties = 
select group_concat(PropertyKey) as column1, 
    group_concat(DataType) as column2, 
    case when group_concat(Alias) <> '' then group_concat(Alias) else null end as column3, -- convert to null to avoid duplicates in union
    group_concat(Description) as column4
from @splitFileLines 
group by OriginalIndex
;

-- 2. Add new properties HERE:

@newProperties = 
values 
('Instrument/ibor/Rating', 'Decimal', 'Rating', 'An instrument rating'),
('Instrument/ibor/Country', 'Text', 'Country', 'The country of issue on an instrument');

-- 3. Join new and old properties

@updatedProperties = 
select * from @currentProperties
union
select * from @newProperties;

-- 4. Write new and old properties into LUSID

@inlinePropertiesInstrument = use Sys.Admin.File.SaveAs with @updatedProperties
--path=/config/lusid/factories/
--type:Csv
--fileNames
instrumentequityproviderfactory
enduse;

-- 5. View system properties which have been saved

select Content from Sys.File wait 5 where Name like '%instrumentequityproviderfactory%';


#### Error handling details to file

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- Write errors to file on Drive
-- ============================================================

@@file_name = select 'daily_instruments';
@@error_path = select 'test/instrument-upload/load-error';
@@processed_path = select 'test/instrument-upload/processed';

@instruments_for_upload =

select
'Amazon Inc' as DisplayName,
'USD' as DomCcy,
'Equities' as AssetClass,
'Equities' as SimpleInstrumentType;

-- Upload the transformed data into LUSID
@writer_results = select *
from Lusid.Instrument.SimpleInstrument.Writer
where ToWrite = @instruments_for_upload;

@writer_status = select count (*) as ResultCount from @writer_results  where WriteErrorDetail is not null;

@target_file_result =

select iif(ResultCount = 0,
@@processed_path,
@@error_path)
as 'FileResult'
from @writer_status;

@@target_dir = select FileResult from @target_file_result;

@save_to_drive = use Drive.SaveAs with @writer_results, @@file_name, @@target_dir
--path=/{@@target_dir}
--fileNames={@@file_name}
enduse;

select * from @save_to_drive;



#### Iif and case when statements

In [ ]:
%%luminesce

-- =============================================================
-- Description:
-- This example shows you how to use iif and case when statements
-- Set a limit, which will decide if the file is a large or small
-- file
-- ==============================================================

@@row_limit = select 10;

-- Load data from LUSID Drive

@data = use Drive.csv
--file=/luminesce-examples/model_portfolios.csv
enduse;

-- Store scalar which is the count of rows in a file

@row_count = select count(*) as [RowCount] from @data;

-- Run an if statement to decide where to save files

@@file_name = select iif(RowCount > @@row_limit, 'large-file', 'small-file') as [TargetFileLocation]
from @row_count;

-- This can also be achieved with a `case when` statement

@@file_name_with_case = select case when RowCount > @@row_limit then 'large-file' else 'small-file' end as [TargetFileLocation]
from @row_count;

-- Save files to drive

@save_to_drive = use Drive.SaveAs with @data, @@file_name
--path=/luminesce-examples
--fileNames={@@file_name}
enduse;

select * from @save_to_drive;


#### Load one cell of data to table by delimiters

In [ ]:
%%luminesce

-- =============================================================
-- Description:
-- Load one cell of data to table by delimiters
-- ==============================================================

@a = use Drive.RawText
--file=/luminesce-examples/DataInOneCell.txt
enduse;

@@content = select Content from @a limit 1;

@raw =
select
    sRow.[Index] as RowIdx,
    sCol.[Index] as ColIdx,
    sCol.Value
from
    tools.split sRow
    inner join tools.split sCol
        on sRow.[Index] = sCol.OriginalIndex
        and sCol.Original = sRow.Value
where
    sRow.DelimiterString = '=@=@='
    and sRow.SplitThisAlone = @@content
    and sCol.DelimiterString = '''~'''
order by 1, 2
    ;

@pivoted =
use Tools.Pivot with @raw
--key=ColIdx
--aggregateColumns=Value
--columnNameFormat="f_{key}{aggregate}"
enduse;

select * from @pivoted


#### Pivot data

In [ ]:
%%luminesce

-- =========================================================================
-- Description:
-- 1. This query show you how to pivot data using the "use Tools.Pivot" tool
-- =========================================================================

-- Load a file of model allocations from Drive
-- The data is structured as one row per portfolio-sector-location allocation

@model_portfolios_orginal = use Drive.csv
--file=/luminesce-examples/model_portfolios.csv
enduse;

-- Transform the data

@model_portfolios_formatted =
select model_port_name as [ModelPortfolioCode],
weighting as [AllocationPercentage],
(sect || "_" || loc) as [AllocationTarget]
from @model_portfolios_orginal;

-- Pivot the data so the model portfolio codes are returned as columns

@model_portfolios_pivoted  =
use Tools.Pivot with @model_portfolios_formatted
--key=ModelPortfolioCode
--aggregateColumns=AllocationPercentage
enduse;

select * from @model_portfolios_pivoted;



#### Random prices generator

In [ ]:
%%luminesce

@@volatility = select 0.5;

@days = 
select date(#2024-01-01#, '-' || Value || ' days', '+1 days') as [Date]
from Tools.[Range] where number = 1000
and Start = 0;

@data = 
select [Date], CASE WHEN RANDOM() % 2 = 0 THEN @@volatility ELSE (@@volatility * -1) END AS random_sign from @days;

SELECT [Date],
       1.2 + SUM(0.01 * random_sign) OVER (ORDER BY [Date]) AS rate
FROM @data;



#### To utc and iso

In [ ]:
%%luminesce

--================================================================

-- Description:
-- In this query we demonstrate uses cases of the to_utc function
-- which takes a string or datetime, with a timezone, and returns
-- a UTC adjusted timezone string.

--================================================================

@data = use Drive.csv
--file=/luminesce-examples/to-utc-datetimes.csv
enduse;

select 

-- Convert ISO strings to UTC

to_utc('2023-01-01', 'Europe/London') as stringDateAsUtc1,
to_utc('2023-07-01 11:00:31', 'Europe/London') as stringDateSummerAsUtc1,
to_utc('2023-01-01 10:00:00', 'US/Eastern') as stringDateAsUtc2,
to_utc('2023-07-01', 'US/Eastern') as stringDateSummerAsUtc2,

-- Convert datetimes to UTC

to_utc(#2023-07-01#, 'Europe/Paris') as datetimeToUtc,


-- Convert non ISO strings to UTC
-- Intermediate conversion completed with the to_date function, which uses the following format
-- https://learn.microsoft.com/en-us/dotnet/standard/base-types/custom-date-and-time-format-strings

to_utc(to_date(Date1 || ' ' || Time1, 'MM/dd/yyyy htt'), 'US/Pacific') as usFormatDateTime,
to_utc(to_date(Date2 || ' ' || Time2, 'yyyy-MM-dd HH:mm'), 'Europe/Dublin') as utcFormatDateTime,
to_utc(to_date(Date3 || ' ' || Time3, 'dd/MM/yyyy h:mmtt'), 'Europe/London') as ukFormatDateTime,

-- Convert datetimes to ISO strings

to_iso(#2022-03-23#) as isoDateTimeString1,
to_iso(#2022-03-23 14:23:45#) as isoDateTimeString2

from @data;


#### Using datetimes

In [ ]:
%%luminesce

--============================================================

-- Description:
-- In this file we show some examples of working with dates in
-- Luminesce. We show creating datetime objects. And also 
-- converting datetime objects to strings.
-- For more information, see official sqlite page on working 
-- with dates:
-- https://www.sqlite.org/lang_datefunc.html

--============================================================

@dt_examples =

select

-- Generate date and datetime objects

date("2025-01-16") as DateObject,
date('Now') as DateObjectNow,
datetime() as DateTimeObject, -- defaults to "Now"

-- Generate strings from datetime objects

strftime('%Y-%m-%d', date()) as DateString,
strftime('%Y-%m-%dT%H:%M%SZ', datetime()) as DateTimeIsoString,
strftime('%Y-%m-%d %H:%M%S', datetime('now', '+7 days')) as DateTimeNextWeek,

-- Generate datetimes from strings
-- to_date can accept strings in any of these formats:
-- https://learn.microsoft.com/en-us/dotnet/standard/base-types/custom-date-and-time-format-strings

to_date("13/02/2023", "dd/MM/yyyy") as DateFromUkFormattedString,
to_date("02-13-2023", "MM-dd-yyyy") as DateFromUsFormattedString,
to_date("13/02/2023 14:23:45", "dd/MM/yyyy HH:ss:mm") as DateTimeFromUkFormattedString,
to_date('2022-01-01 3AM', 'yyyy-MM-dd htt') as StringWithAmToDatetime,
to_date('2022-01-01 3PM', 'yyyy-MM-dd htt') as StringWithPmToDatetime;


select * from @dt_examples;

